In [7]:
# Imports

import os
import h5py
import sqlite3
import pandas as pd

In [8]:
# Variables

track_metadata = "D:\\522\\AdditionalFiles\\track_metadata.db"
artist_term = "D:\\522\\AdditionalFiles\\artist_term.db"
search_dir = "D:\\522\\data"

In [22]:
# Import track metadata

con = sqlite3.connect(track_metadata)
metadata_df = pd.read_sql_query("SELECT track_id,title,release,song_id,artist_id,artist_name,artist_mbid,year from songs", con)
print(metadata_df.head())
con.close()

             track_id              title  \
0  TRMMMYQ128F932D901       Silent Night   
1  TRMMMKD128F425225D        Tanssi vaan   
2  TRMMMRX128F93187D9  No One Could Ever   
3  TRMMMCH128F425532C      Si Vos Querés   
4  TRMMMWA128F426B589   Tangle Of Aspens   

                                release             song_id  \
0                 Monster Ballads X-Mas  SOQMMHC12AB0180CB8   
1                           Karkuteillä  SOVFVAK12A8C1350D9   
2                                Butter  SOGTUKN12AB017F4F1   
3                               De Culo  SOBNYVR12A8C13558C   
4  Rene Ablaze Presents Winter Sessions  SOHSBXH12A8C13B0DF   

            artist_id       artist_name                           artist_mbid  \
0  ARYZTJS1187B98C555  Faster Pussy cat  357ff05d-848a-44cf-b608-cb34b5701ae5   
1  ARMVN3U1187FB3A1EB  Karkkiautomaatti  8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9   
2  ARGEKB01187FB50750    Hudson Mohawke  3d403d44-36ce-465c-ad43-ae877e65adc4   
3  ARNWYLR1187B9B2F9C       Yerb

In [10]:
# Import artist term

con = sqlite3.connect(artist_term)
artist_df = pd.read_sql_query("SELECT * from artist_term", con)
print(artist_df.head())
con.close()

            artist_id          term
0  AR002UA1187B9A637D   garage rock
1  AR002UA1187B9A637D  country rock
2  AR002UA1187B9A637D     free jazz
3  AR002UA1187B9A637D            oi
4  AR002UA1187B9A637D    space rock


In [23]:
# Join between metadata and artist terms

merged_df = pd.merge(metadata_df, artist_df, on=["artist_id"], how='inner')

In [27]:
# Filter category = 'pop'

filtered_df = merged_df[merged_df['term'] == ("pop")]
filtered_df = filtered_df[filtered_df['year'] > 1989]

In [28]:
filtered_df.count

<bound method DataFrame.count of                     track_id  \
1836      TRMMMKD128F425225D   
1861      TRMLAVN128F4252261   
1886      TRWUBYW128F4252258   
1911      TRGBNVG128F425224D   
1936      TRBZRME128F425225E   
1961      TRFOSNY128F4252255   
1986      TRFXTTP128F4252259   
2011      TRQGNIX128F425224F   
2036      TRQRDVN128F425225F   
2061      TRQJXBW128F425225C   
2086      TRABGDV128F4252252   
2111      TRNSZOC128F425224E   
2136      TRULYSN128F425225A   
2161      TRSJKBK128F4252256   
2186      TRVVNZU128F4252251   
2211      TRKWTAK128F4252253   
4925      TRFUCUI128F42955CF   
6617      TRTSRFA128F4238CBE   
7087      TRLMXPD128F4238E12   
8168      TRXKNOO128F4238CAD   
8262      TRYAUUS12903CA4D9D   
8435      TRMMMBB12903CB7D21   
8481      TRMCIDU12903CB7F26   
8527      TRMOSDO12903CB7D20   
8619      TRGAOQI12903CB5B92   
8665      TRGPHCU12903CB5B38   
8711      TRGTWIJ12903CB81C3   
8757      TRGSEIO12903CB7D1C   
8803      TRHWFDD128F93430BF   
8849   

In [14]:
# Search for H5 data files on hdd
# Dataset comes with songs in hundreds of different folders but can't find a database that
# links tracks to folder locations. Must manually do this
# Its a lengthy process so recommend saving the df to hdd for caching

h5_files = {}
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".h5"):
             h5_files[file.split('\\')[-1][:-3]]=os.path.join(root, file)

In [15]:
# Convert dict to DF
cache_location = pd.DataFrame([[key,value] for key,value in h5_files.items()],columns=["track_id","location"])

In [16]:
# Save to file
cache_location.to_csv("cache_location.csv")

In [7]:
# Read DF in
cache_location = pd.read_csv('cache_location.csv')

In [29]:
# List of songs that should be analyzed
songs_to_copy = pd.merge(filtered_df, cache_location, on=['track_id'], how='inner').drop_duplicates(subset=['location'])

In [30]:
# Python get rid of special characters
types = songs_to_copy.apply(lambda x: pd.lib.infer_dtype(x.values))
for col in types[types=='unicode'].index:
    songs_to_copy[col] = songs_to_copy[col].apply(lambda x: x.replace('[^\x00-\x7F]','').decode('unicode_escape').encode('ascii', 'ignore').lower().strip())

In [31]:
songs_to_copy.count

<bound method DataFrame.count of                   track_id                                              title  \
0       TRMMMKD128F425225D                                        Tanssi vaan   
1       TRMLAVN128F4252261                                    Yeah yeah Jenni   
2       TRWUBYW128F4252258                                          Äl-oo-vee   
3       TRGBNVG128F425224D                                      Jää beibi jää   
4       TRBZRME128F425225E                                    Takaisin en tuu   
5       TRFOSNY128F4252255                                Paina kaasua_ Honey   
6       TRFXTTP128F4252259                               Ei oo ei oo toivokaa   
7       TRQGNIX128F425224F                              Aina vaan jaa jaa jaa   
8       TRQRDVN128F425225F                                   Annathan anteeks   
9       TRQJXBW128F425225C                                    Taaskin turhaan   
10      TRABGDV128F4252252                                Toivon että huomaa

In [32]:
songs_to_copy.to_csv("song_metadata.csv")

In [48]:
songs_to_copy.dtypes

track_id       object
title          object
release        object
song_id        object
artist_id      object
artist_name    object
artist_mbid    object
term           object
location       object
dtype: object